
# Fine-tuning BART for summarization in two languages

---


This notebook contains an example of fine-tuning [Bart](https://huggingface.co/transformers/model_doc/bart.html) for generating summaries of article sections from the [WikiLingua](https://huggingface.co/datasets/wiki_lingua) dataset. WikiLingua is a multilingual set of articles. We will run the same code for two Bart checkpoints, including a non-English model from the [Hugging Face Model Hub](https://huggingface.co/models). We will be using:
- the **English** portion of WikiLingua with [sshleifer/distilbart-xsum-12-3](https://huggingface.co/sshleifer/distilbart-xsum-12-3) Bart checkpoint, and
- **French** articles from the WikiLingua with the [moussaKam/barthez-orangesum-abstract](https://huggingface.co/moussaKam/barthez-orangesum-abstract) model.

## Setup

---

In [1]:
! pip install transformers
! pip install datasets
! pip install sentencepiece
! pip install rouge_score
! pip install wandb

     |████████████████████████████████| 4.0 MB 13.1 MB/s 
     |████████████████████████████████| 895 kB 45.1 MB/s 
     |████████████████████████████████| 596 kB 47.4 MB/s 
     |████████████████████████████████| 6.6 MB 43.0 MB/s 
     |████████████████████████████████| 77 kB 6.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 325 kB 8.0 MB/s 
     |████████████████████████████████| 1.1 MB 20.6 MB/s 
     |████████████████████████████████| 136 kB 49.2 MB/s 
     |████████████████████████████████| 212 kB 52.1 MB/s 
     |████████████████████████████████| 127 kB 50.5 MB/s 
     |████████████████████████████████| 94 kB 3.5 MB/s 
     |████████████████████████████████| 271 kB 50.5 MB/s 
     |████████████████████████████████| 144 kB 52.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urlli

In [2]:
import torch
import numpy as np
import datasets

from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
)

from tabulate import tabulate
import nltk
from datetime import datetime

In [3]:
WANDB_INTEGRATION = True
if WANDB_INTEGRATION:
    import wandb

    wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


## Set language

---

Pick a language. In our example, there is a choice of English and French.

French

In [4]:
language = "french"

English

In [5]:
language = "english"

## Model and tokenizer

---

Download model and tokenizer. Use default parameters or try custom values (see [HF Bart configuration](https://huggingface.co/transformers/_modules/transformers/configuration_bart.html) and [Fairseq Bart](https://github.com/pytorch/fairseq/tree/master/examples/bart)).

In [6]:
model_name = "sshleifer/distilbart-xsum-12-3"
if language == "french":
    model_name = "moussaKam/barthez-orangesum-abstract"

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set model parameters or use the default
# print(model.config)

# tokenization
encoder_max_length = 256  # demo
decoder_max_length = 64

Downloading:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/683M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

## Data

---

### Download

For demonstration, we are only using a small portion of the data.

In [7]:
data = datasets.load_dataset("wiki_lingua", name=language, split="train[:2000]")

# Take a look at the data
for k, v in data["article"][0].items():
    print(k)
    print(v)

Generating train split:   0%|          | 0/57945 [00:00<?, ? examples/s]

Dataset wiki_lingua downloaded and prepared to /root/.cache/huggingface/datasets/wiki_lingua/english/1.1.1/dad5f0710d1bb1a2ed1eb7633802fe147ebd27a3812985f5641bb7aca6ba5c08. Subsequent calls will reuse this data.
section_name
['Finding Other Transportation', 'Designating a Driver', 'Staying Safe']
document
['make sure that the area is a safe place, especially if you plan on walking home at night.  It’s always a good idea to practice the buddy system.  Have a friend meet up and walk with you. Research the bus, train, or streetcar routes available in your area to find safe and affordable travel to your destination.  Make sure you check the schedule for your outgoing and return travel.  Some public transportation will cease to run late at night.  Be sure if you take public transportation to the venue that you will also be able to get home late at night. Check the routes.  Even if some public transit is still running late at night, the routing may change.  Some may run express past many of 

### Prepare

**Format and split into train and validation sets**

In [8]:
def flatten(example):
    return {
        "document": example["article"]["document"],
        "summary": example["article"]["summary"],
    }


def list2samples(example):
    documents = []
    summaries = []
    for sample in zip(example["document"], example["summary"]):
        if len(sample[0]) > 0:
            documents += sample[0]
            summaries += sample[1]
    return {"document": documents, "summary": summaries}


dataset = data.map(flatten, remove_columns=["article", "url"])
dataset = dataset.map(list2samples, batched=True)

train_data_txt, validation_data_txt = dataset.train_test_split(test_size=0.1).values()

  0%|          | 0/2000 [00:00<?, ?ex/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

**Preprocess and tokenize**

In [9]:
def batch_tokenize_preprocess(batch, tokenizer, max_source_length, max_target_length):
    source, target = batch["document"], batch["summary"]
    source_tokenized = tokenizer(
        source, padding="max_length", truncation=True, max_length=max_source_length
    )
    target_tokenized = tokenizer(
        target, padding="max_length", truncation=True, max_length=max_target_length
    )

    batch = {k: v for k, v in source_tokenized.items()}
    # Ignore padding in the loss
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in l]
        for l in target_tokenized["input_ids"]
    ]
    return batch


train_data = train_data_txt.map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer, encoder_max_length, decoder_max_length
    ),
    batched=True,
    remove_columns=train_data_txt.column_names,
)

validation_data = validation_data_txt.map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer, encoder_max_length, decoder_max_length
    ),
    batched=True,
    remove_columns=validation_data_txt.column_names,
)

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

## Training

---

### Metrics

In [10]:
# Borrowed from https://github.com/huggingface/transformers/blob/master/examples/seq2seq/run_summarization.py

nltk.download("punkt", quiet=True)

metric = datasets.load_metric("rouge")


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract a few results from ROUGE
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
    ]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

### Training arguments

In [11]:
training_args = Seq2SeqTrainingArguments(
    output_dir="results",
    num_train_epochs=1,  # demo
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=4,  # demo
    per_device_eval_batch_size=4,
    # learning_rate=3e-05,
    warmup_steps=500,
    weight_decay=0.1,
    label_smoothing_factor=0.1,
    predict_with_generate=True,
    logging_dir="logs",
    logging_steps=50,
    save_total_limit=3,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_data,
    eval_dataset=validation_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

### Train

Wandb integration

In [12]:
if WANDB_INTEGRATION:
    wandb_run = wandb.init(
        project="bart_wiki_lingua",
        config={
            "per_device_train_batch_size": training_args.per_device_train_batch_size,
            "learning_rate": training_args.learning_rate,
            "dataset": "wiki_lingua " + language,
        },
    )

    now = datetime.now()
    current_time = now.strftime("%H%M%S")
    wandb_run.name = "run_" + language + "_" + current_time

wandb: Currently logged in as: megha067 (use `wandb login --relogin` to force relogin)


Evaluate before fine-tuning

In [13]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 484
  Batch size = 4


Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


{'eval_gen_len': 23.1901,
 'eval_loss': 6.764211654663086,
 'eval_rouge1': 20.832,
 'eval_rouge2': 4.6858,
 'eval_rougeL': 15.2072,
 'eval_rougeLsum': 18.2032,
 'eval_runtime': 145.8575,
 'eval_samples_per_second': 3.318,
 'eval_steps_per_second': 0.83}

Train the model

In [14]:
#%%wandb
# uncomment to display Wandb charts

trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 4351
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 1088


Step,Training Loss
50,6.529400
100,5.672100
150,5.150500
200,4.843300
250,4.846200
300,4.716400
350,4.724300
400,4.622800
450,4.630600
500,4.607100


Saving model checkpoint to results/checkpoint-500
Configuration saved in results/checkpoint-500/config.json
Model weights saved in results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in results/checkpoint-500/tokenizer_config.json
Special tokens file saved in results/checkpoint-500/special_tokens_map.json
Saving model checkpoint to results/checkpoint-1000
Configuration saved in results/checkpoint-1000/config.json
Model weights saved in results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in results/checkpoint-1000/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1088, training_loss=4.71834470244015, metrics={'train_runtime': 973.9984, 'train_samples_per_second': 4.467, 'train_steps_per_second': 1.117, 'total_flos': 1346978041036800.0, 'train_loss': 4.71834470244015, 'epoch': 1.0})

Evaluate after fine-tuning

In [15]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 484
  Batch size = 4


{'epoch': 1.0,
 'eval_gen_len': 23.1839,
 'eval_loss': 4.248395919799805,
 'eval_rouge1': 31.2606,
 'eval_rouge2': 12.1048,
 'eval_rougeL': 25.1015,
 'eval_rougeLsum': 30.0078,
 'eval_runtime': 161.0085,
 'eval_samples_per_second': 3.006,
 'eval_steps_per_second': 0.752}

In [16]:
if WANDB_INTEGRATION:
    wandb_run.finish()

eval/gen_len,█▁
eval/loss,█▁
eval/rouge1,▁█
eval/rouge2,▁█
eval/rougeL,▁█
eval/rougeLsum,▁█
eval/runtime,▁█
eval/samples_per_second,█▁
eval/steps_per_second,█▁
train/epoch,▁▁▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇███
train/global_step,▁▁▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▆▇▇▇███


## Evaluation

---

**Generate summaries from the fine-tuned model and compare them with those generated from the original, pre-trained one.**

In [17]:
def generate_summary(test_samples, model):
    inputs = tokenizer(
        test_samples["document"],
        padding="max_length",
        truncation=True,
        max_length=encoder_max_length,
        return_tensors="pt",
    )
    input_ids = inputs.input_ids.to(model.device)
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return outputs, output_str


model_before_tuning = AutoModelForSeq2SeqLM.from_pretrained(model_name)

test_samples = validation_data_txt.select(range(16))

summaries_before_tuning = generate_summary(test_samples, model_before_tuning)[1]
summaries_after_tuning = generate_summary(test_samples, model)[1]

loading configuration file https://huggingface.co/sshleifer/distilbart-xsum-12-3/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/4a0f7fb35f1504b6e865136124e3781fb488792aa105a84a991a3145a027791f.10ebe969457e130b9da526e7994b6191d3765d1d01ac6abc2eb20bb8adcbd4e0
Model config BartConfig {
  "_name_or_path": "sshleifer/distilbart-xsum-12-3",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 3,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_t

In [18]:
print(
    tabulate(
        zip(
            range(len(summaries_after_tuning)),
            summaries_after_tuning,
            summaries_before_tuning,
        ),
        headers=["Id", "Summary after", "Summary before"],
    )
)
print("\nTarget summaries:\n")
print(
    tabulate(list(enumerate(test_samples["summary"])), headers=["Id", "Target summary"])
)
print("\nSource documents:\n")
print(tabulate(list(enumerate(test_samples["document"])), headers=["Id", "Document"]))

  Id  Summary after                                                                                                           Summary before
----  ----------------------------------------------------------------------------------------------------------------------  --------------------------------------------------------------------------------------------------------------------------
   0  Make a case to protect your keys. Replace your batteries every few years.                                               Batteries for your Mercedes keys and keychain can be difficult to replace.
   1  Get a collar and microchip. Teach your cat to use a litter box. Teach it to use the litter.                             When you bring a cat home and name it, you should get it some identification.
   2  Understand the quality of your work. Go over the course syllabus. Consider whether it’s worth the hassle.               It is important that you go over the syllabus before approaching your professor

## Examples

It appears that for both languages, after a short fine-tuning, our model has adopted the imperative mode of target summaries.

### English

~~~
  Id  Summary after                                                                                                                                      Summary before
----  -------------------------------------------------------------------------------------------------------------------------------------------------  ----------------------------------------------------------------------------------------------------------------------
   0  Change your network’s name. Determine how many people would use the guest network.                                                                 If you’re using your home’s “guest” network to connect to your wireless network, you may have the option to change it.
   1  Be the first person to arrive at the site at the time of the event. Make a checklist for your crew.                                                As the world gears up for an event, we take a look at some of the tips you need to make.
   2  Eat fermented dairy products. Eat non-dairy fermented foods. Take a probiotic supplement.                                                          Here is a guide to the best ways to restore your gut flora.
   3  Consider what you need to bring. Bring clothes. Take a few music.                                                                                  If you are looking for a way to get away from home, what do you bring?
   4  Buy a bra that fits right. Check the bra band. Check your bra band is parallel to the ground or lower.                                             A bra that doesn't fit right will look "off" under your clothes.
   5  Get your feelings out of the way. Write a letter for your friend or partner. Talk about what you like about the person.                            If you're breaking up with a partner or friend, it's best to get your feelings out of the way.
   6  Place the child’s diaper on the child's waist. Insert the diaper into the diaper. Close the pin.                                                   It’s almost time for you to fix a new type of diaper pin.
   7  Use a dry bar of soap to fill in your nail holes.                                                                                                  Do you use soap to fill in your nail holes?
   8  Measure the width at the bottom of the shower door. Cut the rail at the appropriate length. Place the bottom track along the base of the opening.  If you want to build a shower on the back of the door, you need to be careful.
   9  Choose a paint that is suitable for metal. Apply a self-etching primer. Spray the primer onto the surface of the brass.                            If you’re a fan of brass, it’s important to know what to do if you paint the metal.
  10  Wash your skin with warm water. Do not touch your eyes, mouth or genitals. Rinse your blisters in a bath.                                          If you've come into contact with poison ivy, you might want to wash your skin.
  11  Wear your hair. Look for a fairy-like look.                                                                                                        If you are especially fair, apply a rosy blush to your cheeks and lipstick to look for a fairy look, what do you do?
  12  Be a permanent resident in the UK. Show that you have enough income to provide support for your parents.                                           If you are legally staying in the UK on a semi-permanent basis, you can likely bring your parents over.
  13  Be aware of the dangers of filling balloons.                                                                                                       If you are making your own way of filling balloons, you might want to think twice before using the process.
  14  Be aware of the dangers of alcohol and substance abuse. Be aware of mental health issues.                                                          If you are a college student, you might be thinking carefully about your college years.
  15  Pass the papers to a substitute. Mail the papers by first-class mail. Send a copy of all court papers by substitution.                             When you serve court papers by substitution, you need to do it to the respondent.

Target summaries:

  Id  Target summary
----  --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
   0  Click on Guest Access. Allow Guest Access. Identify a guest network name. Define a guest password. Define Total Guests Allowed. Allow SSID Broadcast. Save your settings.
   1  Set everything up. Delegate. Be flexible. Keep everyone updated. Do your thing. For marketing events, give a takeaway. Congratulate yourself after the event. Clean up and get out! Take care of returns, payments, and thank all parties.
   2  Consume fermented dairy products. Add non-dairy fermented foods to your meals 1 to 2 times per day. Take a probiotic supplement.
   3  Prepare a list of things to bring. Consider the weekend's activities. Pack to suit the weather. Keep possible emergencies in mind. Pack lightly.
   4  Make sure your bra is the right size. Choose styles that provide adequate support for large breasts. Check online if you're having trouble finding a bra in your size.
   5  Write a letter to yourself first if it's a bad goodbye. Focus on the good times. Talk about what you like about the other person. Apologize and forgive the other person. End with a kind sentiment if you can.
   6  Fold the diaper up and over until the diaper's top center is at the child’s waistline. Flare out and position both sides of the back portion of the diaper a little (if this diaper is not a folded/elasticated one). Secure the diaper in place with diaper-pins or Velcro.
   7  Rub a bar of soap over the holes to fill them without tools.
   8  Cut the tracks to the appropriate size, if necessary. Measure and mark where the track will go. Drill the holes you marked. Install the bottom threshold track. Mount the wall tracks. Install the door swinging door. Measure and cut the top track. Seal any gaps using caulk.
   9  Choose a spray paint in your desired color. Apply a coat of primer. Apply several thin coats of paint. Apply a protective clear coat.
  10  Wash your skin. Avoid touching the rash. Soak in an astringent bath. Soak in a bath. Use cool compresses.
  11  Create fairy hair. Brighten your complexion. Use make-up to create a natural, fairy look. Dress like fairy.
  12  Have permanent status or citizenship. Ensure your parents need long-term care. Save enough money to support your parents for 5 years.
  13  Observe safety precautions while filling balloons.
  14  Avoid alcohol and substance abuse. Stay alert. Take care of your mental health.
  15  Know who else you can give the papers to. Instruct the substitute. Mail another copy. Fill out a Proof of Service form.

Source documents:

  Id  Document
----  ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
   0  There is no need to change any of your basic network or wireless settings. Choose "Yes" from the options. Usually “guest” is just appended on the end of your current network’s name. Some routers may allow you to change this name. Just make sure it’s not the same name as your current network. Since you’re technically creating a new network, you have to define a corresponding network password.  It is a good idea not to use the same password of your home network. You may have the option to define how many you would allow to use the guest network at any given time.   Fewer people using your network would mean better quality of connection for everyone on the network. Remember, your actual bandwidth won’t increase and everyone would just share. You may opt to have this guess network broadcasted or hidden.
   1  Be the first to arrive at the site to oversee the preparations. Make sure everyone files in accordingly and start making phone calls if not. Assist those who need assisting, direct those who need directed, and get out of the way when you need to. No injuries until after the event, please.  You'll feel a little calmer if you make a checklist. Have a portion for your crew, a portion for hired out crews, a portion for decorations and basic set-up, and a portion for equipment. When all is checked off, that's when you have time to breathe. Do not be afraid to delegate. The main stressor when it comes to an event is time. To save it, all hands need to be on deck -- or different decks. If someone isn't being as useful as they could be, give them something better to do. It's your job. It's not you being bossy or overstepping your boundaries; it's you doing what you're supposed to do.  When you delegate, be firm but polite. Say, "Juan, I need you over here right now to help out with the caterers. Thank you." Your crew needs to jive together as much as the event does. Keep everything moving swimmingly by being the leader that you need to be. This means making sure that things are kept to schedule and assisting or having backup plans if it falls behind -- and being okay with it. If you start stressing out, you'll lose your head. Nothing good will come of that. So when that speech runs ten minutes over and the speaker is ignoring your blatant distress signals or your attempt at feigning a stroke to create a diversion, relax. You'll just adjust the appetizer and no one will notice. Event intact. Things happen. They do. They have a bad habit of happening. There's no way you can predict everything, and the sooner you accept this the better. A calm, collected event manager can do wonders for any event; a strung-out, tense one cannot. So relax and go with the flow -- it'll all be over soon! Check that guest numbers and needs are still correct on the day of the event and advise the service team of any changes at the earliest possible opportunity You should be the first set of eyes that notices if anything goes wrong.  Consult with your clients to see how they are feeling; they may be excited, nervous, worried, bored or mentally drained or have some issue on their plate that in some cases you may soothe with understanding, some kind words and practical assistance. It is wise to use this time to rejuvenate the enthusiasm in guests and the team where appropriate. Trust and respect your service team to look after themselves -- if you've given them a good base, they should be fine. Offer assistance if needed, but they should be experienced enough not to need any help at all. Here are a few things to keep in mind:   Act as a concierge or receptionist at the beginning, meeting and greeting each guest (if appropriate) as they come. Hand over the reins to the MC when the event starts. The management role will be more active problem solving and ensuring all the back of housework such as food preparation and service runs to plan. Keep an eye on the guests and keep contact with the MC often and discreetly in case they want (or need) to change plans. Keep a respectful distance from the key guests - after all, the event is all about them - but be easily accessible by checking at appropriate moments how they feel the event is going, as well as any problems, requests or suggestions they may have. You want all your guests to remember how awesome of a time they had. Actually, you probably want more than that. A visit to a website, future donations, word of mouth buzz, whatever. To ensure that your event is the legend it deserves to be, give a takeaway. Whether it's a picture, a flyer, or a pen, having something to remind your guests of the event is a good way to make sure you stay in their minds. Most events tend to run themselves when they start, but all the hard work is the preparation that no-one sees. So pat yourself on the back because you deserve it! Okay, back to tearing it down. The job's not over yet!  After the event, arrange a time to meet and thank your client. It is always recommended to offer an appropriate and thoughtful gift to remember their time with you, as it is these small touches that make the experience richer and may make them recommend your services in future. If you gave a gift during the event, such as in a gift registry with the other guests, then a thoughtful after present such as flowers, a framed photograph of your favourite moment at their event (such as cutting the ribbon, or the climax of the show, or the award ceremony, or the wedding kiss, or blowing out the candles on a cake, etc), or some other gift may be appropriate. Just like mom said, "Leave it in the same condition you found it in," the same goes for your venue. Everything needs to be just like it was before you got there -- this is one business where you don't want to burn any bridges. So let your crews know it's time to tear down and don't let them leave until all is taken care of. And you need to pitch in too!  This is nice, sure, but it also prevents you from being billed any more than you should be. Many places will tack on extra cleaning fees if they find any opportunity to do so. So make it as spic and span as possible to avoid the hidden costs. You may need to arrange returning hired or borrowed equipment and later on consulting with the client about their experience. Even if unpaid after the event, thank them for the opportunity to have such a great experience to run an event with them. Can you get a business card of theirs?  Thank your crew, too! Make sure all parties are paid (and all parties have paid up), file receipts, and get everyone taken care of. You should be one of the last ones out the door -- and make sure it's locked behind you.
   2  Fermented dairy products contain both probiotics and prebiotics. Eat 1 serving of fermented dairy products per day. Fermented dairy products include plain yogurt, cottage cheese, kefir, whey, buttermilk, leben, and crème fraîche.  Look for yogurt products with the label “active cultures” on it. Choose natural yogurts over sweetened yogurts. Instead, sweeten your yogurt naturally with fruit like bananas, peaches, blueberries, and strawberries. Probiotics are microorganisms that stimulate the growth of gut flora. Prebiotics are the fibers that feed probiotics. Non-dairy fermented foods, or pickled foods, are a great alternative to fermented dairy products if you are lactose intolerant. Like fermented dairy products, pickled foods contain probiotics. Eating pickled foods consistently will help you restore your gut flora. Non-dairy fermented foods include sauerkraut, kimchi, corn relish, miso, tempeh, natto, fermented tofu, and pickled vegetables like cucumbers, beets, cabbage, and radish. Taking a probiotic supplement is another great way to restore your gut flora. Take the supplement per the instructions on the bottle. If you are already eating fermented foods to restore your gut flora, it is safe to take a probiotic supplement as well. However, you can take a supplement in place of eating fermented foods, if you do not like fermented food.  Contact your doctor if you are unsure how often you should take the supplement, or if you are taking other medications. If you are taking an antibiotic, a probiotic supplement may help restore your gut flora.  Probiotic supplements are formulated to treat many different conditions. If you have IBS, for instance, you might try using VSL#3. Talk to your doctor or your pharmacist about the ideal product for your specific symptoms.
   3  A to-bring list of items can vary widely depending on the type of weekend trip you're planning. Certain things, like an extra set of clothes, are recommended across the board, while other things can be brought or left depending on the type of place you're going to. Here is a list of things to consider bringing:   Clothes. For a weekend trip, a fresh set of clothing should be considered necessary. You will want to feel fresh on the second and third day of being away from home, and in the case that your first set of clothes gets wet or dirty, you'll want something to fall back on. The same applies to socks and underwear. Bring heavier clothing, like a jacket, based on the forecasted weather.   Toiletries. Toothpaste and a toothbrush should be deemed essential for your trip. If the place you're going has running water, a small vial of shampoo and conditioner should be brought along as well to ensure you're feeling fresh throughout the trip.  Trip entertainment. The journey to and from your location should be just as enjoyable as when you get there. This can be helped by taking along some entertainment. If you're driving, bring a few of your favourite CDs to cycle through. A good book is also recommended, particularly if you're going somewhere relaxing like a lake.  Trip-applicable items. A weekend vacation is often built around a certain activity. Your to-bring list should mention the specific things you'll need to perform that activity. If you're fishing, for example, you will need a rod and a bait box, among other things. Depending on where you're going, items like some mild first aid should be considered as well. A weekend trip is relatively easy to outline. It's not a large amount of time, and it's usually focused on one specific activity. Try to gauge what you will need. Can you think of any activities you'll most likely be doing? What are you hoping to do over the weekend? Running through the vacation in your mind's eye will give you a stronger idea what you should be bringing along with you.  For example, if you're going to a lake in the Summer, useful items such as bug spray and lighter fluid are recommended. Imagine the possible scenarios that relate to the environment in question, and speculate as to the things you would need. If there are activities you may do but aren't sure about (like swimming, for example) it's best to bring them along anyway. A weekend trip isn't going to require much baggage to begin with, and you might feel left out if you decide not to bring something. Your packing should be arranged with respects to the weekend's activity and the environment it's taking place in. Because your trip will only encompass a couple of days, it's prudent to check the weather forecast for those days and plan accordingly. If there's going to be rain, the clothes you bring should reflect that. A poncho and umbrella will be sorely missed if you neglect to check the weather beforehand. In the case of sun, bringing sunglasses and sunscreen will be helpful. Cold weather demands you bring heavier clothing, like a winter parka, along for the ride. If you're somehow unsure, prepare for the worst just in case.  Even if your weekend trip will be spent mostly indoors, bringing an umbrella can make a major difference in the case of rain. Download an app like Weatherbug for on-the-go weather forecasts. Under the best circumstances, you won't need to use everything you bring with you on a weekend trip. In case something bad happens however, you don't want to be far from home and wishing you had brought something. This is especially true if you're going to be camping or otherwise hiking somewhere where help isn't immediately available.  Carry a small bit of first-aid along with your toiletries. This includes some band-aids, aloe vera and headache relievers. These will take up a negligible amount of space, and you'll be glad you brought them if it comes time to use them. A set of extra socks will come in handy, especially if the weather is forecasted to be rainy. Nothing spoils a good weekend faster than wet feet. Even if you're only going to be gone for a weekend, an extra set of underwear is never a bad idea. For a weekend trip, you're more likely to overpack than underpack. Unless you're going camping, you won't need much to hold you over for a weekend and carrying too much stuff around will sap some of the enjoyment of travel to begin with. If you can, see how much you can minimize the amount of stuff you bring, and try to stuff it all into a backpack. You don't want to skimp out on things you might miss, but optional luxuries like a hair dryer can probably be done without over a 48 hour period. For a weekend trip, try to limit yourself to a single backpack forth of clothing and items. This will force you to prioritize what you need over the things you can do without. One change of top is usually all you'll need for a weekend.
   4  A bra that doesn't fit right will look "off" under your clothes. You may be spilling out of the cups or have a band that is too tight and looks lumpy, which may draw attention to your chest. A bra that provides proper support and fits correctly will help you feel comfortable and may improve your confidence.  Check that your bra straps don't dig into your shoulders. Your straps should not be working hard to support your breasts. You may need a smaller band size or a bra with wider straps. You can also buy padding that slips under your straps if your bra fits right but your straps still hurt. Make sure the bra band (the strap that runs across your back) is parallel to the ground or lower. The bra band should provide most of the support, and the lower it is on your back, the more support it can provide.  The cups should be smooth against your chest. If your breasts are spilling out the top or sides or it feels like your bra is cutting into your breasts, try a larger cup or a bra with a bigger cup design.  You should be able to fit a finger under the front of your bra band. If it's too tight, you need a larger band size.  A professional fitting can be extremely beneficial in helping you find a bra that fits correctly. An employee at a lingerie store or department store can measure you for the right band and cup size. The wrong bra can make your breasts look saggy but, even worse, can cause you significant pain. There are many different bra types on the market, and you'll need to find out which ones work for your body.  Try a multi-cup bra. The cups on these bras are made up of two or more pieces of fabric and they provide a ton of support. They will center your breast tissue, which will slenderize your silhouette.  Try a long-line bra, which can relieve stress on your shoulders and back. The band on this bra is extended down to your waist, which distributes the weight of your breasts more evenly over your body. It can also act as shape wear, giving you a smooth line under your clothes.  Try a minimizer bra, especially if you like button-up shirts. Minimizer bras minimize your breasts' projection, or how far they stick out. If you find your breasts cause gaps between the buttons in your shirts, a minimizer will push them back. Be aware that this can also create cleavage, which you may want to avoid if you're trying to deflect attention from your breasts. If you find that the stores in your area don't have bras that accommodate women with larger breasts, there are plenty of great online retailers that offer attractive bras that will give you the support you need. Keep in mind that if you are above a DD cup, sizing will vary among different companies. Check the sizing on the website to make sure you choose the right bra.
   5  That is, if you're breaking up with a partner or letting go of a toxic friend, it's best to get your own feelings out of the way first. Take time to write a healing letter and list all your complaints and hurt feelings. Use this letter to vent everything, and then you can move on to writing a letter for your friend or partner. The first letter tends to be emotionally charged. You may want to blame the other person for everything or hurl insults. But getting it out of the way is good because then you can move on to a calmer, more rational letter. Spend at least part of the letter talking about what you like about the person and your relationship together. In most relationships, there are good times as well as bad, and no matter why you're saying goodbye, you can still end on a good note by taking time to think about what was good. You could write about good memories, like how you met or when you fell in love. Even though you're saying goodbye to the person, you can still note what you liked about the person in the past. You don't need to trash the person to move on, and if you really feel the need to, then maybe that's a letter you should keep to yourself instead. For example, you could say, "I love your kind spirit, your willingness to go on adventures, and your compassion for everyone you met. You made me want to be a better person, and for that, I will always be grateful." If you're breaking up, you both likely played some part in how ended. Don't hold back. Apologize for any part you played in what happened, and if you can, offer forgiveness to the other person. That way, you don't part with bad blood between you. For example, you might write, "I'm sorry for how things have ended between us, and I apologize for my part in that. I don't hold any grudges against you, and I hope you won't hold any against me." If this person was special to you, you can make note of that at the end. Discuss how you'll still think about the person in the future, for instance, and how you'll remember the good times fondly, despite the fact that you need to go your separate ways. For instance, you might say, "I'm sorry that we have to go our separate ways, but I think we just don't work together anymore. No matter what, you will always hold a special place in my heart."
   6  Hold the diaper firmly as you move it around and place it properly. Lay the dry diaper temporarily on the child’s waist so that you can obtain the new pins or get ready to fix the new diaper up again. The amount of diaper you flare back out at the bottom will depend on the child’s weight and width of their waistline. Only flare out and work with one side of the child's diaper at a time. You can repeat this for the other side later. Grab any two pins that aren't in use yet and secure the diaper, reversing the pin's removal from the preparatory steps above. Insert the pin into the diaper and close the pin. Do not push each pin through all the layers of the inner diaper, so you don't risk sticking yourself and your baby. Be sure to insert the end of the pins  away  from the child’s navel/belly button. Snap the diaper pins closed from both sides. Diaper pins are created with an open section in the pin-head. Push the pin around and into the head of the pin’s opening, and push inwards so the pin-head is covering this end of the pin.
   7  Rub a dry bar of soap over your nail holes, back and forth. Stop once the holes look filled in. Run a dry cloth or rag over the surface of the holes to wipe excess soap pieces away.  Avoid using a wet or damp rag, which can melt or dissolve the soap. Opt for a white soap if you have white walls. You can use this method for either drywall or plaster walls.
   8  Measure the width at the bottom of the shower opening. Transfer that measurement to the bottom shower door track and mark with your marking pen. If the track fits to size, move forward with installation. If it's too long, you'll need to cut it to the appropriate length. Using a hacksaw, carefully cut the rail at the mark you made. Be sure the rail is held firmly as you cut to avoid damage to the rail or the saw. Use a file to remove the burrs from all metal pieces you cut. Before you go attaching the tracks on forever, you need to place them temporarily and mark where they need to go. Place the bottom track along the base of the shower opening with the higher lip of the track facing outward. Make sure the track sits flat along the surface. There should be about 1⁄8 inch (0.3 cm) of play at each end.  Temporarily tack the bottom track into place with masking tape, then mark the position with your marking pen along the inner and outer edges. Do not remove the bottom track yet. The wall tracks should be pre-cut to size from the factory. Slide the wall tracks into position with the bottom track. Make sure the wall tracks fit the bottom track exactly. Use your level to check for plumb. Holding each track firmly, use your marking pen to mark the mounting hole locations on each wall then set the wall tracks aside. With a nail or a center punch, tap a small dimple into the marks you made for the holes in the wall tracks to use as a pilot hole. This prevents your drill bit from “skating” and damaging the surface. Drill your mounting holes using the proper bit for the surface of your shower. If you are drilling into tile, put a small piece of masking tape over each spot to be drilled. This will help prevent chipping. Also, drill deep enough so the plastic screw anchors will fit properly. Anchors won't be needed on fiberglass. Run a bead of caulk, about the thickness of a bead of toothpaste, along the bottom mounting surface. Center the bead between the two lines you marked when you measured and run it the full length of the gap as well. Then, firmly set the bottom track into place over the bead of caulk.  Make sure the underside of the track makes contact with the caulking. If not, run a separate bead along the center of the underside of the track. Hold the track in place for a minute or two, taping it down to hold it in place if necessary. It should be try in five minutes at the most, then you can move forward knowing that it's secure. Line them up with the mounting holes and make sure they fit properly over the ends of the bottom track. If you've measured correctly and marked correctly, they should snap right into place. If your kit included them, place the rubber bumpers that come with most door kits over the screws and secure the tracks to the wall by using your screwdriver to turn the screws into place. Do not completely tighten the screws at this point, hand tight is sufficient. Swinging doors will need to be installed differently, according to the kit that you've purchased, so it's important to read the instructions that come with it and follow the directions accordingly. The door needs to be installed so it pivots outward, but for some kits that'll be on the left side and for some that'll be on the right side, and the mechanism works differently, depending. With some, the door will just snap into place, while screws will be used for other kits. In most swinging door kits, a strip of rubber will be slid into the wall track opposite the pivot point, held with screws in some cases. If you had to cut the threshold track, you'll likely also have to cut the top track, since it'll be roughly the same length. Make sure the track fits snugly, connecting between the two wall tracks, and is properly aligned between them. It should just snap right on top. Many door kits will have corner brackets that are attached with screws to hold the top rail securely. Defer to the instructions of your specific kit, if necessary. Finally, run a bead of bathtub caulk along all the points at which the tracks come in contact with the walls. Do this to both inside and outside surfaces to create a neat, watertight seal. Let the caulk dry thoroughly and wait at least 24 hours before running water in the shower to test your work. It should be dry in a few minutes, but it's a good idea to wait and let things settle before you try it out.
   9  Pick a paint that is suitable for metal, such as an enamel paint, an acrylic- or oil-based product, or another hard-drying paint. Most paints that are suitable for metal will be spray paints, but some may come in liquid (roll-on) form. Avoid latex paints for brass, as these don’t adhere as well to metal and aren't as durable. They may be usable if you have selected a high quality primer. The best primer for brass is a self-etching primer, also known as a bonding primer. This is a mixture of acid and zinc, and it will adhere better to the brass than any other type of paint or primer. Shake the primer well and hold it six to eight inches (15 to 20 cm) from the surface of the metal. Spray in a sweeping motion, going from side to side, and apply a thin, even coat of primer onto the surface.  Let the primer dry for about 24 hours, or according to the manufacturer’s instructions. Wear all proper safety equipment when using spray paint, including gloves, safety glasses, and a mask. Even after scrubbing with steel wool, brass still doesn’t have a great surface for painting, which is why self-etching primer is important. Once the primer has had time to dry, apply the paint in the same way. Shake the can, use a sweeping side to side motion, and hold the can six to eight inches (15 to 20 cm) from the surface to apply a thin, even coat.  Let each coat dry completely according to the manufacturer’s instructions (usually an hour or two) before applying a second or third coat.  Depending on the effect you're trying to achieve, you may need anywhere between two and five coats of paint. If your paint came in a liquid form, use a brush or roller to apply thin, even coats. Once the paint has had time to fully dry—usually about 24 hours—you can apply a clear top coat to finish the piece. This will help seal the piece, protect the paint, and make the finish shiny. Look for clear coats or enamel coats that are safe for metal.  Shake the can and hold it six to eight inches (15 to 20 cm) from the surface. Spray the coat in a back and forth motion to apply an even coat on the metal. Set the piece aside to dry according to the manufacturer’s instructions. These coats tend to dry quickly, sometimes in as little as 30 minutes.
  10  As soon as you realize that you've accidentally come into contact with poison ivy, wash your skin very thoroughly. Use lots of warm, soapy water. If you can, wash within the first 30 minutes of touching the poison ivy. If you're still outdoors, find a stream or creek. Place the affected area in the running water for at least 10 minutes.  You should also wash underneath your fingernails. If you're washing up at home, wash all of your clothes and shoes or boots as well. A poison ivy rash can easily spread by simply scratching or touching. If you've come into contact with poison ivy or have a rash, do not touch anywhere around your eyes, mouth, or genitals. All parts of a poison ivy plant (even dead plants) contain an oily allergen called urushiol. This causes a rash or blisters if it comes into contact with your skin or you breathe it in. If you do get a rash around your eyes, mouth or genitals, you should see your doctor immediately. If you have blisters from the poison ivy, never break them open since this increases your risk of infection and scarring. Instead, soak your blisters in a bath with a solution of Burow's solution. You can buy a product containing this solution of aluminum sulfate and aluminum acetate at a drugstore or pharmacy. Soak the blisters for 20 minutes at least two or three times a day. Soaking in Burow's solution acts as an astringent which can reduce the size of the blisters and help dry them out. Fill a sock or nylon knee-high with steel-cut oatmeal. Tie the filled sock onto the tap of your bathtub. Run a cool bath so that the water runs through the oatmeal and into the tub. Soak in the oatmeal bath as long and as often as you like.  Studies have shown that oatmeal is great for soothing your rash and relieving itchiness. The less you scratch your rash, the sooner it will dry up.  You can also purchase an oatmeal bath product that you simply mix into your tub. Soak a clean cotton towel in cold water and squeeze out the extra water. Place the cold cloth over the rash for as long as it stays cool. When it warms up, simply run the cloth under cold water and wring it out again. You can do this as often as you like.  To make an astringent compress that dries out the rash, brew a pot of tea. Soak a clean towel in the cooled tea and apply it to your rash. When your body temperature is higher, your rash may seem itchier. Applying a cool compress can make the rash less itchy and soothe your skin.
  11  Keep your hair natural and whimsical. Messy bohemian styles best capture the fairy look.  If you have long hair, wear it down or in braids. For those who prefer shorter hair, consider a pixie cut or a messy bob. Fairy-like details include piece-y (not perfectly even) bangs and tousled hair. Adorn your hair with ribbons and flowers. Antique pins and barrettes also make for a nice fairy look. Even out your complexion with a light-weight tinted moisturizer. If you are especially fair, apply a rosy blush to your cheeks. Apply a shimmering powder to your cheekbones. Use a highlighter pencil under your brow bone. Fairy beauty is fresh, glowy and wide-eyed.  Wear lip-gloss or a sheer lipstick. Look for products with shimmer. Avoid matte lipsticks, as these are too heavy.  Good color choices include pink, red and wine. Widen your eyes with mascara and light eye-shadow. Choose a good mascara and use only one coat to avoid heavy, spidery lashes. Look for light, shimmering shadows. Mocha, silver and moss-green are some good color options. Fairy fashion is feminine and ethereal, old-fashioned and romantic.  Wear long skirts and flowy dresses. Look for gossamer fabrics and filigree detailing. Err on the side of demure. In summer, you can show your arms, shoulders and perhaps some décolleté, but avoid hems that are too high (think thigh-high) and plunging necklines. For your winter wardrobe, check out the winter fashions of 19th-century Russia. The long coats, muffs and round hats capture the romantic fairy look. Accessorize with headbands and jewelry. Again, antique designs, beads, ribbons and flowers create a romantic and whimsical fairy look. Fairies are often barefoot, but this is not practical. Flat shoes, sandals and Mary Janes are excellent choices. Low-heeled shoes are acceptable, but exceedingly high, stiletto heels are not compatible with the free and easy fairy look.
  12  For your parents to come over, you must be a permanent resident in the U.K. That means that you need to have citizenship, permanent leave to remain, permanent residency, refugee status, or humanitarian protection. Basically, if you are legally staying in the UK on a semi-permanent basis, you can likely bring your parents over. The main way you can bring over your parents is if they need assistance with daily care tasks, including household tasks, personal hygiene tasks, and/or medical tasks. Your parents must need this care because of illness, disability, or age.  Also, they must not be able to get this care where they are living right now, which includes not having any relatives there who can provide that support. Exceptions may be made if care is not affordable. Both parents must apply at the same time if they are both still alive and still married. In other words, if one parent could depend on the other for care, the UK government won't allow just the one parent who needs care to apply. Both parents must need care. In other words, you'll need to show you have sufficient income to provide continuing support for your relatives. You must not use public funds to provide this support.  Generally, you must prove you can at least meet the threshold of making 18,600 pounds per year, which is the amount where you are not dependent on the state for things like a housing credit or tax credits.  It can help to have money saved. If you need to, try creating a budget to help you put some money back.
  13  Use caution when performing the pouring and cutting required in filling party balloons. Adults should supervise or perform parts of this process for children.  Use scissors carefully when cutting balloons. You can also avoid spills of the filling by laying down newspaper or another covering over your work area. Take care if you have a latex allergy, in which case you should opt for balloons made of Mylar or another material instead of the common latex. Adults should handle balloons for infants and toddlers, as balloons and balloon pieces are a choking hazard.
  14  The percentage of alcohol and substance abuse is at its highest during college years. While a legal drink or two might be okay, know when you need to stop. Never take any substance and drive. No matter what situation you are in -- be it a party or walking around campus -- be alert to your surroundings. Watch for suspicious individuals or actions.  When walking around campus, always take well-walked routes and avoid walking around alone at night. Check out your campus safety's website. Often, campuses have emergency phones (they are often blue) scattered around campus for use. Almost half of college students reported feelings of depression or anxiety while in college. Check out your college campus resources for mental health.  Often, campuses offer free or cheap mental health care. Talk to a trusted person (advisor, professor, resident advisor) if you are feeling down. They can help point you to helpful resources.
  15  If the server has done all in his or her power to serve the papers directly to the respondent and has been unable to do so, your server can hand those papers over to a party that can legally accept them on the respondent's behalf. A competent adult, age 18 or older, who lives at home with the respondent can usually accept the papers. Similarly, an adult who appears to be in charge at the respondent's workplace or an adult who seems to be in charge where the person receives mail can usually accept the papers, as well. When passing the court papers to a substitute, your server must leave that substitute with specific instructions regarding what the papers are and what needs to be done with them.  Make sure that the substitute knows that he or she is receiving court papers for a specific individual. The respondent must be named, and the substitute must be told to deliver the papers to the respondent. Get the substitute's name when you leave the papers. If the substitute refuses to provide a name, write down a thorough physical description of the substitute. When you serve papers by substitution, you must also mail another copy of all court papers by first-class mail. Address the package to the respondent.  You need to send the papers to the same address the server left the papers at. Afterward, the server must fill out the "Proof of Service" form as usual. In addition to this form, a "Proof of Mailing (Substituted Service)" form must also be filled out by the server.  Get both forms from the server after he or she fills them out. File them both together at the courthouse on or before your court date. Provide a postal receipt with your forms indicating that you sent a second copy of the papers through the mail.


~~~

### French

~~~
  Id  Summary after                                                                                                  Summary before
----  -------------------------------------------------------------------------------------------------------------  ---------------------------------------------------------------------------------------------------
   0  Sachez que le harcèlement n'a pas de rapport avec vous. Sachez que vous ne                                     Le harcèlement est une des causes principales de la dépression chez les adolescents. Cependant, il
   1  Apprenez à développer les compétences de base nécessaires au sport. Apprenez à jouer avec la balle             Apprendre à maîtriser la balle avant de la lancer et la retourner est indispensable pour devenir un
   2  Créez un groupe d'amis. Jouez avec vous. Jouez avec vous.                                                      Alors que vous êtes en train de baiser, un groupe d'amis est en préparation
   3  Essayez de faire rire quelqu'un. Essayez de faire rire quelqu'un. Essayez                                      Tester son sérieux en conversation. C'est le genre de méthode qui marche pour
   4  Submergez le crâne dans un grand container rempli d'eau. Enr                                                   Les animaux dont le crâne est utilisé en préservation sont des carnivores. Ils possèdent une
   5  Mixez les ingrédients. Ajoutez le sel et le poivre. Ajoutez le petit-lait                                      C'est parti pour un tour de magie. Avec cette recette, vous allez faire
   6  Trouvez une clé de sol. Trouvez une clé de sol. Trouvez une clé de sol                                         Les notes se notent en fonction de la position du morceau, selon le système de notation
   7  Parlez d'un ami commun. Parlez de vos expériences. Parlez de vos expériences.                                  Apprendre à parler de vous, de votre famille, de vos amis... C'est
   8  Créez un disque. Choisissez le type de média que vous voulez créer. Choisissez le type                         Vous n'avez pas besoin de tous les 4 disques durs vierges et vous pouvez utiliser
   9  Donnez beaucoup d'attention aux miaulements de votre chat. Donnez-lui de                                       Les miaulements de votre félin peuvent être variés. Ils expriment souvent un besoin
  10  Évitez de voir la méduse sur le rivage. Évitez de toucher la méd                                               Les sauveteurs en mer sont formés pour venir au secours des baigneurs dans des situations
  11  Choisissez un moment précis pour lire. Faites des pauses.                                                      Vous n'avez pas besoin de toujours lire au même endroit et vous pouvez le faire
  12  Utilisez une présentation informelle. Utilisez des mots simples. Utilisez des exemples. Utilisez des exemples  Pour renforcer votre CV et votre crédibilité, optez pour une présentation informelle. Rédigez une
  13  Faites des recherches. Faites de la publicité. Faites de la publicité. Faites de la                            Pour pouvoir réaliser un documentaire, il est conseillé de solliciter l'aide d'un
  14  Apprenez à vous connaitre. Apprenez à vous connaitre. Apprenez à vous connaitre. Apprenez à                    Vous avez toujours voulu être un homme de caractère, mais vous avez toujours voulu être un
  15  Mélangez le lait et la vanille. Ajoutez le mélange. Ajoutez le pouding à la                                    Vous pouvez remplacer l'extrait de vanille par de l'extrait d'amandes pour

Target summaries:

  Id  Target summary
----  ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
   0  Comprenez pourquoi le harcèlement se produit. Restez maitre de la situation. Signalez la situation à un adulte de confiance. Changez de point de vue. Obtenez le soutien dont vous avez besoin. Soyez actif.
   1  Développez les compétences de base. Apprenez les responsabilités liées à votre position. Entraînez-vous beaucoup. Restez en bonne santé. Hydratez-vous bien pendant les entraînements. Écoutez votre entraîneur. Communiquez sur le terrain. Continuez malgré la douleur.
   2  Réunissez en groupe vos amis. Asseyez-vous en formant un cercle. Faites tourner la bouteille ! Embrassez la personne pointée par la bouteille. Passez au joueur suivant et faites tourner encore. Amusez-vous !
   3  Posez des questions. Répétez ce que vous allez dire. Pensez à des situations similaires.
   4  Faites-le tremper dans de l'eau oxygénée. Replacez les dents dans leurs alvéoles. Utilisez du coton sur les dents de carnivores. Séchez le crâne. Préservez avec du polyuréthane.
   5  Mixez les épices. Assaisonnez la farine. Mélangez l'œuf et le petit-lait.
   6  Regardez une portée. Apprenez les notes des lignes. Apprenez les notes des interlignes. Dépassez la portée. Lisez l'armure. Déterminez la mesure. Distinguez chaque mesure. Apprenez les figures de notes.
   7  Mentionnez un ami commun. Parlez d'une expérience partagée. Posez une question intéressante. Mentionnez un centre d'intérêt commun. Posez une question personnelle.
   8  Utilisez l'outil fourni par le fabricant de votre ordinateur. Téléchargez un fichier ISO de Windows 7. Copiez le contenu du DVD d'installation sur votre disque dur. Téléchargez le Kit d'Installation Automatisée Windows. Téléchargez et installez NTLite. Cliquez sur le bouton Ajouter. Doublecliquez sur la version de Windows 7. Sélectionnez Drivers dans le menu. Téléchargez tous les pilotes « manquants ». Cliquez sur le bouton Ajouter. Sélectionnez Post-installation. Automatisez l'installation (facultatif). Cliquez sur Appliquer. Cochez la case Créer l'ISO. Cliquez sur Processus. Faites un clic droit sur le fichier ISO.
   9  Identifiez ses miaulements. Interprétez les miaulements. Observez le langage corporel du chat.
  10  Ne touchez pas aux méduses qui ont échoué sur le rivage. Nagez à proximité d'un sauveteur . Éloignez une méduse. Sortez immédiatement de l'eau si vous voyez ou soupçonnez la présence d'une méduse.
  11  Fixez-vous une période de lecture. Emportez votre livre avec vous. Lisez tous les jours. Prenez des notes. Ne remettez pas au lendemain. Faites-vous aider. Trouvez-vous un pote de lecture. Appréciez vos lectures !
  12  Choisissez le ton de la conversation. Présentez-vous. Apportez une dimension personnelle. Soyez bref.
  13  Recrutez une équipe si nécessaire. Pensez aux affinités. Apprenez les techniques de base de réalisation. Équipez-vous. Planifiez, organisez et exposez les grandes lignes de votre tournage.
  14  Écrivez votre histoire. Analysez votre histoire. Décidez de ce que signifie votre analyse. N'oubliez pas que cela prend du temps.
  15  Mélangez le pouding, le lait et la vanille.

Source documents:

  Id  Document
----  -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
   0  Les adolescents qui harcèlent les autres le font en général parce qu'ils ont des problèmes dans leur propre vie. Malheureusement, ils ont des difficultés à gérer leurs propres problèmes, c'est pourquoi ils projettent leur tristesse sur vous. Cependant, il est important de savoir que le harcèlement n'a pas de rapport avec vous. Vous avez de belles qualités, quoi que cette personne puisse vous dire. Elle vous harcèle probablement à cause d'une des raisons suivantes  :   le désir de se sentir puissant la jalousie le besoin de prouver aux autres qu'elle est forte le besoin de se sentir fort la recherche d'un exutoire à sa douleur intérieure une réponse au harcèlement qu'elle subit elle-même La chose la plus simple est de vous éloigner de la personne qui vous harcèle. Vous pouvez aussi rester où vous êtes et l'ignorer. Autrement, vous pouvez prendre votre défense en lui disant calmement que vous n'êtes pas intéressé par ce qu'il dit. La chose la plus importante à ce moment-là est de garder votre sang-froid. Vous ne voulez pas avoir une réaction trop émotionnelle et prendre le risque d'y répondre de manière agressive.  En répondant à la personne qui vous harcèle avec humour, vous deviendrez souvent une cible moins intéressante. Les réponses amusantes font souvent perdre leur intérêt aux personnes qui harcèlent les autres, ce qui signifie qu'il ne viendra plus vous embêter. Assurez-vous de rester en sécurité. Ce n'est pas parce que vous répondez sans être agressif que vous devez vous mettre dans une situation où vous n'êtes pas en sécurité. S'il vous blesse physiquement, vous pouvez vous protéger pour vous échapper d'une situation où vous prenez un risque pour votre sécurité . Si cette personne n'est pas confrontée à ses actes, elle va devenir de plus en plus agressive envers vous. Il est très important d'en parler à un adulte de confiance pour éviter que la situation dégénère .  Soyez implacable pour que cette personne arrête de vous harceler. Signalez chaque incident jusqu'à ce qu'il s'arrête. N'ayez jamais honte de demander de l'aide. Vous n'êtes probablement pas la seule personne que cet individu harcèle, mais vous pouvez aider à mettre fin à son comportement. La plupart du temps, les adultes peuvent trouver des solutions au problème sans dire à cette personne que vous les avez informés de son comportement. Certaines solutions pourraient impliquer que vous changiez de classe ou que vous changiez de place en classe. La personne qui vous harcèle pourrait aussi recevoir des punitions. Si vous êtes le témoin du harcèlement de quelqu'un d'autre, vous devez aussi le signaler. Personne ne mérite de se faire harceler. Souvenez-vous que la personne qui vous harcèle est simplement quelqu'un de malheureux qui essaye de vous faire tomber aussi bas qu'elle . Lorsque vous pensez à partir de son point de vue, le harcèlement perd un peu de son pouvoir. Souvenez-vous que vous ne devez pas le laisser contrôler vos émotions.  Dressez une liste de toutes les choses positives chez vous. Vous pouvez y inclure toutes les bonnes choses qui se passent dans votre vie. Chaque fois que vous avez le cafard, vous pouvez relire votre liste. Essayez de ne pas faire empirer la situation en ressassant l'incident et en le repassant en permanence dans votre tête. À la place, concentrez-vous sur les choses positives qui se sont produites pendant votre journée. Assurez-vous de discuter de vos expériences. Même si vous ne voulez pas ressasser le harcèlement toute la journée, il est important de trouver une occasion d'exprimer ce que vous ressentez. Vous pouvez discuter avec un parent, un membre de votre famille, un professeur, un conseiller, un prêtre ou un ami. Vous allez probablement vous sentir mieux en en discutant .  Donnez-vous le temps de guérir. Le harcèlement est une expérience plutôt traumatisante. Il va être utile d'en parler, mais il va vous falloir un peu de temps avant de pouvoir vous sentir de nouveau normal. Vous pouvez demander de l'aide à un professionnel si vous remarquez que vous avez du mal à gérer vos sentiments de colère, vos blessures ou toute autre émotion négatifs. Le bénévolat est une excellente façon de faire disparaitre un sentiment d'impuissance qui pourrait apparaitre lorsque vous êtes la victime de harcèlements . Vous pourriez aider d'autres adolescents ou des enfants plus jeunes qui ont été victimes de harcèlement ou vous pourriez participer à une campagne contre le harcèlement dans votre école. Ce que vous choisissez n'a pas d'importance, cependant il est important de retrouver un sens de contrôle en devenant plus actif.
   1  Si vous voulez être un bon joueur dans votre équipe, vous devez tout d'abord travailler à devenir un bon sportif en général, en passant du temps à développer les compétences de base nécessaires au sport que vous pratiquez. Si vous voulez devenir un bon joueur de basketball, vous devez passer du temps à dribbler, à développer vos compétences en matière de défense et de passes efficaces. Si vous voulez devenir un bon joueur de football, vous devez apprendre à contrôler le ballon, à tirer correctement et à vous déplacer sur le terrain.  Il est très amusant de sortir et d'aller jouer au sport qui vous plaît, mais il est important de vous entraîner. Au lieu de simplement jouer avec la balle, entraînez-vous à dribbler ou répétez des mouvements de défense que votre entraîneur vous a appris. Vous aurez plus de chances de vous démarquer des autres et de devenir un bon joueur si vous travaillez les compétences moins amusantes nécessaires à votre sport. Le sport en équipe présuppose que chacun tient une position particulière. À la différence du tennis ou du golf, vous devez remplir une fonction lorsque vous jouez en équipe. Tous les joueurs lors d'une partie de football ne marquent pas des buts, tous les joueurs de basketball ne mettent pas des paniers. Un bon joueur connaît les responsabilités liées à sa position et a appris à remplir son rôle le mieux possible.  Apprenez la position que vous devez tenir sur le terrain et quelles actions vous devez faire. Si vous jouez en défense, apprenez à choisir un adversaire et à le marquer. Si vous êtes en possession du ballon, apprenez les meilleures façons de faire suivre l'action. Lorsque vous apprenez à jouer à un sport au début, il est normal de vouloir occuper une position plus glorieuse, par exemple attaquant ou libéro. Cependant, une bonne équipe est faite de joueurs qui tiennent les positions les mieux adaptées à leurs compétences. Si vous êtes un bon défenseur, ne gaspillez pas votre énergie en étant jaloux des attaquants. Donnez-vous à fond dans votre rôle et entraînez-vous pour améliorer vos compétences. Il est très important de vous entraîner dur et de donner tout ce que vous avez si vous voulez devenir un bon joueur en équipe. Entraînez-vous sans relâche pour améliorer vos compétences et vos connaissances du sport que vous pratiquez, afin de vous préparer, ainsi que votre équipe, au succès.  Arrivez à l'heure lors des entraînements et préparez-vous à travailler. Préparez votre équipement et beaucoup d'eau pour rester hydraté. Échauffez-vous et préparez-vous à l'entraînement. Vous devez comprendre que l'entraînement est très important. Certains sportifs sont très doués, mais ils se comportent comme s'ils préfèreraient être à la maison devant la console de jeu plutôt qu'avec leurs coéquipiers. Vous pouvez être meilleur que cela. Faites de votre mieux pendant l'entraînement. Si vous ne voulez pas trop forcer alors que vous êtes censé lever des haltères, courir des tours de terrain ou faire des exercices, vous allez être plus lent, plus faible et moins doué que vos adversaires. Développez ces qualités lors de vos entraînements. Même si vous êtes un bon sportif, vous ne pouvez pas être un bon joueur si vous passez votre temps à vous remettre de blessures ou à rester assis sur le banc de touche. Vous devez prendre le plus grand soin de votre corps et restez en bonne santé pour faire face à vos adversaires et donner à votre équipe toutes les chances de victoire.  Échauffez-vous avant l'entraînement et reposez-vous après, à chaque fois. Ne courez jamais sur le terrain sans avoir fait des étirements et des échauffements, pour préparer votre corps à l'effort physique. Les bons joueurs doivent aussi prendre quelques minutes pour s'étirer après l'entraînement, pour éviter les crampes et les irritations. Reposez-vous suffisamment entre vos entraînements. Si vous devez retourner à l'entraînement demain, vous ne devez pas vous coucher au petit matin parce que vous avez passé la nuit à jouer à la X-Box ou à chatter en ligne. Dormez suffisamment, au moins 8 heures par nuit et laissez suffisamment de temps à votre corps pour se remettre et pour se reposer avant de le remettre au travail le lendemain. Une étude sur les joueurs de la NFL a démontré qu'environ 98% d'entre eux étaient déshydratés après les entraînements, ce qui peut conduire à des pertes de performances de 25%. Les boissons énergétiques et l'eau sont des éléments importants qui vous aident à rester hydraté et à conserver les électrolytes dans votre corps, ce qui vous donne ensuite suffisamment d'énergie pour être au top de vos performances et pour rester en bonne santé. Avant l'entraînement, buvez environ un demi-litre d'eau et essayez de boire environ 250 ml d'eau toutes les 15 minutes pendant votre entraînement. Buvez lentement pour éviter de déranger votre estomac pendant ces séances d'efforts physiques intenses. Les bons joueurs doivent être faciles à entraîner, ce qui signifie que vous devez savoir recevoir les critiques et appliquer les nouvelles techniques qui vous sont données pour pouvoir vous améliorer sur le terrain. Le travail des entraîneurs n'est pas de dire aux joueurs qu'ils font du bon travail et que tout le monde deviendra joueurs professionnels. Les entraîneurs sont là pour faire de vous un meilleur athlète et pour vous amener vers la victoire. Parfois, cela signifie que vous devrez entendre des suggestions ou des critiques.  Les mauvais joueurs se referment et sont frustrés lorsqu'ils reçoivent des critiques, alors que les bons joueurs vont écouter et en tirer des leçons. Si votre entraîneur vous rappelle à l'ordre parce que vous ne descendez pas assez pendant que vous faites des pompes, cela peut vous mettre de mauvaise humeur ou vous pouvez répondre Oui coach ! et transpirer un peu plus. Ne vous disputez jamais avec votre entraîneur, surtout en face des autres joueurs. Si vous n'êtes pas d'accord avec la stratégie ou avec quelque chose que votre entraîneur vous a dit pendant l'entraînement, demandez-lui de le rencontrer en privé et parlez-en avec lui. Les bons joueurs ne remettent jamais en question l'autorité de leur entraîneur en face du reste de l'équipe. Les équipes sportives doivent être organisées et coordonnées pour pouvoir gagner. Les équipes silencieuses perdent et les équipes bavardes augmentent leurs chances de gagner. Vous devez vous entraîner à parler plus que les autres équipes. Il est important de parler avec vos coéquipiers, mais évitez d'insulter vos adversaires. À moins que cela devienne nécessaire pour motiver vos coéquipiers. Faites-le, mais faites-le gentiment. Il n'est pas toujours amusant de s'entraîner et le sport que vous pratiquez peut être vraiment fatigant. Mais les bons joueurs (et les excellents joueurs) apprennent à oublier la douleur pendant leurs entraînements et à la combattre. Lorsque vous êtes fatigué à la fin d'une partie et que le ballon se retrouve entre vous et le but, vous pouvez courir après en tirant la langue ou vous pouvez rassembler vos dernières forces et vous lancer dans un sprint. Les bons joueurs courent très vite Trouvez des façons de rester motivé et alerte pendant la partie afin de conserver toute votre énergie et tout votre enthousiasme tout le long de la partie. Passez de la musique forte qui vous donne envie de courir, mettez-vous dans l'ambiance en regardant un film sportif ou faites des exercices de groupes qui vous plaisent.
   2  Lorsque le moment est venu de jouer au jeu de la bouteille, la première chose dont vous aurez besoin sera d'autres personnes (à moins que vous vouliez jouer tout seul, ce qui serait une scène assez triste). Pour commencer, créez un groupe d'amis volontaires. Plus nombreux vous êtes, plus vous vous amuserez ! Essayez de constituer un groupe composé de personnes des deux sexes afin qu'il puisse avoir un éventail de plusieurs combinaisons de personnes potentielles qui pourront s'embrasser. Prenez soin de faire savoir à vos amis de quoi il s'agit avant de commencer le jeu. Un baiser est une chose qui est d'habitude réservée à des personnes qui s'aiment réellement, donc le fait d'obliger des personnes qui ne veulent pas s'embrasser à le faire peut être vraiment embarrassant pour chacune des personnes impliquées. N'obligez personne à jouer avec vous contre son gré. Quand tout le monde est prêt (et se porte volontaire) pour jouer, disposez-vous en cercle de manière à ce que chacun ait la face tournée vers l'intérieur du cercle. D'habitude, on le fait sur le sol, cependant rien n'empêche que vous le fassiez également debout ou assis autour d'une table. Peu importe comment vous faites pour disposer votre groupe en cercle, mais il est généralement préférable de choisir un endroit dont la surface est dure, solide, comme un endroit ayant un plancher en bois. Ceci garantit que la bouteille que vous utiliserez pourra bien tourner sans érafler ou endommager le plancher. Quand vous êtes tous prêts à jouer, choisissez une personne pour commencer le jeu. Cette personne saisira une bouteille (ou, en réalité, tout objet similaire pouvant être tournoyé, comme un bic, un verre, une clé, etc.) et la fera tournoyer fortement au milieu du cercle de joueurs. Après que la bouteille ait commencé à tournoyer, personne ne doit la toucher jusqu'à ce qu'elle s'arrête complètement. Si vous hésitez quant au moyen de choisir celui ou celle qui doit faire tourner la bouteille le premier ou la première, vous pourrez choisir de laisser d'abord jouer les joueurs les plus âgés ou les plus jeunes ou bien vous pouvez demander à quelqu'un d'imaginer un nombre par hasard et commencer le jeu par celui qui imaginera par la suite un nombre qui est plus proche de celui de ce dernier. Lorsque la bouteille s'arrête, son cou (le bout ouvert) doit surement pointer une personne faisant partie du cercle. La personne qui a fait tournoyer la bouteille doit embrasser cette dernière !  Si vous n'utilisez pas une bouteille, alors désignez un des bouts de l'objet, qui sera considéré comme le bout qui servira à pointer les personnes. Par exemple, si vous utilisez un bic, vous pouvez décider d'utiliser le bout qui sert à écrire en tant que sa « pointe ». Si vous faites tourner la bouteille et qu'elle s'arrête sur vous, faites-la tournoyer à nouveau. C'est aussi simple que cela ! Après qu'une personne ait fait tourner la bouteille et ait donné un baiser à la personne pointée à la fin par la bouteille, la personne qui est assise à côté de lui ou d'elle dans le cercle fait tournoyer la bouteille et donne un baiser à celui ou celle que la bouteille pointe à la fin de son action. Opérez les tournoiements dans un sens unique, soit dans le sens de déplacement d'une montre ou dans le sens contraire.  Notez que parfois la bouteille s'arrêtera de tourner en pointant l'espace entre deux personnes, dans ce cas, vous devez décider que celui qui a fait tournoyer la bouteille embrasse la personne la plus proche du champ de pointage de la bouteille. Également, la bouteille peut pointer une personne que vous n'avez pas envie d'embrasser en raison de vos préférences sentimentales (par exemple, si vous êtes attiré(e) par des garçons et que le pointage tombe sur une fille). Dans ce cas, vous pouvez choisir d'embrasser la personne du sexe approprié qui est installée le plus proche du champ de pointage de la bouteille. Félicitations ! Vous avez appris comment jouer au jeu de la bouteille. Ce jeu amusant et facile est plus intéressant lorsqu'il se joue dans une attitude joviale de la part de tous les joueurs. Ne laissez pas tourner les choses dans une ambiance trop romantique ou sexuelle, c'est juste un jeu, mais il peut devenir très vite assez embarrassant s'il est trop pris au sérieux. Toutefois, si vous ressentez une étincelle significative avec une personne après le jeu, il n'y a aucune raison qui vous empêcherait de chercher à la connaitre « après » le jeu ! Le jeu de la bouteille est très approprié pour déclencher de nouvelles amitiés que vous n'auriez pas pu avoir en temps normal.
   3  Si vous essayez de garder votre sérieux pendant que vous discutez avec quelqu'un, engagez-vous encore plus dans la conversation. Vous pouvez y arriver par exemple en posant des questions. C'est une excellente façon de diriger la conversation vers un sujet qui ne vous fait pas rire .  Par exemple, si la personne avec laquelle vous discutez parle de son travail, mais si elle glisse vers un sujet que vous trouvez drôle, vous pouvez essayer de lui poser des questions qui vont la rediriger vers le sujet de son emploi. Vous pourriez dire par exemple : « pourrais-tu m'en dire plus à propos de ton parcours professionnel ? » ou bien « comment t'es-tu lancé dans ce domaine ? » Si vous avez du mal à garder votre sérieux pendant que vous racontez une blague ou que vous jouez à un jeu, entrainez-vous en avance. Répétez la blague autant de fois que nécessaire jusqu'à ce que vous puissiez la raconter en gardant votre sérieux. Vous pouvez aussi vous entrainer à répéter les choses dont vous allez parler pour vous distraire pendant le jeu et garder votre sérieux.  Si vous essayez de penser à autre chose pendant un jeu, discutez d'un sujet qui n'a aucun rapport. Par exemple, si vous jouez au poker, ne discutez pas d'autres parties de poker auxquelles vous avez participé. Parlez plutôt de ce qu'il se passe au travail ou d'un incident qui s'est produit pendant que vous étiez au supermarché. Les gens avec lesquels vous discutez vont se sentir écoutés si vous étayez ce qu'ils disent avec des expériences personnelles similaires. En réfléchissant à votre passé, vous allez oublier le présent et vous allez mieux arriver à contrôler votre visage. Par exemple, si quelqu'un vous raconte une histoire embarrassante qui a l'air de lui faire un peu de mal, vous pouvez aussi partager une histoire embarrassante. Cela va vous empêcher de rire et aider l'autre personne à se sentir mieux dans la conversation.
   4  Submergez le crâne dans un grand container rempli d'eau. Incluez seulement 300 à 450 ml de peroxyde d'hydrogène à 35 % pour environ 5 l d'eau .  N'utilisez pas d'eau de javel, car cela risque d'endommager les os et les dents. Cela va blanchir le crâne. Les crânes sont naturellement d'une couleur blanche crème à jaunâtre. Laissez le crâne dans l'eau pendant au moins 24 heures. Si vous avez utilisé une méthode à base d'eau pour nettoyer le crâne, les dents vont tomber des mâchoires de la bête. Gardez les dents et remettez-les en place à l'aide d'une petite quantité de colle forte . La plupart des animaux dont le crâne est utilisé en préservation sont des carnivores. Ils possèdent donc une paire de canines. Ces canines ont tendance à être beaucoup plus petites que les alvéoles auxquelles elles sont fixées. Utilisez des boules de coton trempées dans de la colle. Enroulez la dent dans un peu de coton et insérez le tout dans l'alvéole. Laissez le crâne 24 heures à l'extérieur pour le sécher au soleil et laisser la colle prendre. Le crâne ne contiendra plus aucune matière organique. Vous n'aurez donc plus à vous inquiéter des animaux ou des insectes. Vaporisez plusieurs couches de polyuréthane sur le crâne. Laissez chaque couche sécher avant d'appliquer la suivante. Cela va laisser une finition lisse et brillante.
   5  Broyez tous les assaisonnements dans un mixeur jusqu'à ce que les particules fassent toutes la même taille. Les ingrédients à moudre sont le sel de cèleri, le sel à l'ail et le sel nature ainsi que les herbes séchées, le poivre noir et blanc moulu, la moutarde en poudre et le paprika. Mixez-les jusqu'à obtention d'une poudre homogène.  Pour obtenir une saveur plus authentique, ajoutez également une cuillère à café de sel aux herbes par kilogramme de poulet. Si vous ne trouvez pas d'origan ou de thym, essayez de la marjolaine et de la sauge. Versez-la dans un cul-de-poule et ajoutez les épices moulues. Mélangez les ingrédients avec un fouet jusqu'à ce que la poudre soit homogène. Mettez le cul-de-poule de côté pour le moment. Vous allez y mettre les morceaux de viande. Le récipient doit être suffisamment grand pour accommoder au moins un blanc ou une cuisse de poulet. Battez-les ensemble dans un récipient à part. Ne les ajoutez pas à la farine assaisonnée. Cassez l'œuf dans un grand cul-de-poule et ajoutez le petit-lait ou le yaourt. Fouettez les deux ingrédients jusqu'à ce que le mélange soit homogène et qu'il ne reste plus de trace de jaune d'œuf. Assurez-vous que le récipient est suffisamment grand et profond pour que vous puissiez immerger complètement tous les morceaux de poulet.
   6  Il s'agit de cinq lignes horizontales superposées fermées aux deux extrémités. À gauche, vous trouverez une clé (de sol pour une partition de guitare), deux numéros superposés, par exemple 44{\displaystyle {\frac {4}{4}}} et parfois une série de bémols ou de dièses. Toutes ces indications sur la portée donnent les informations relatives au rythme et à la tonalité du morceau.   Un morceau pour guitare peut également se noter sous forme de tablature, système de notation musicale particulier fait pour la guitare. Une tablature ressemble à une portée traditionnelle, mais comprend six lignes au lieu de cinq, aucune clé et souvent le mot « tab » écrit sur la gauche. Les partitions pour guitare se notent toujours en clé de sol, ce qui signifie que le symbole tout à fait à gauche de la portée est une clé de sol (un G ancien très stylisé) formant une boucle autour de la deuxième ligne à partir du bas . Chaque ligne de la portée correspond à une note, comme un mi, un sol, etc. Lorsque vous voyez une figure de note sur la partition, vous devez jouer la note à laquelle sa position correspond, mais pour cela, vous devez savoir de laquelle il s'agit. En allant de bas en haut, les notes qui correspondent aux lignes de la portée sont mi, sol, si, ré et fa. Les espaces entre les lignes de la portée correspondent également à des notes. La portée comprend ainsi neuf notes de base (sans compter celles altérées par des dièses ou des bémols, que nous aborderons plus tard). En allant de bas en haut, les notes qui correspondent aux interlignes sont fa, la, do et mi. En prenant l'ensemble des lignes et des interlignes, la portée entière avec toutes les notes correspond au schéma ci-dessous :   fa mi ré do si la sol fa mi Des traits supplémentaires au-delà de la portée principale permettent d'étendre la gamme de notes que l'on peut écrire. Si vous voyez une figure de note avec un petit trait horizontal au-dessus ou au-dessous de la portée, il s'agit d'une note plus aigüe ou plus grave que celles qui se trouvent sur les lignes et les interlignes de base.  Chacun de ces petits traits permet d'ajouter deux notes, une avec la tête de note à hauteur du trait et une en dessous, mais pour le moment, apprenez simplement les bases. Elle vous permettra de déterminer la tonalité du morceau. L'armure est un ensemble d'altérations qui se trouve entre la clé et le chiffrage de mesure au début de la partition. Elle comprend une série de dièses (♯), de bémols (♭) qui se suivent sur la portée. Pour connaitre les tonalités qui correspondent aux différentes armures, vous devez simplement les mémoriser. Cependant, même si vous ne connaissez pas la tonalité, l'armure peut vous donner des informations très utiles.  En fonction de la ligne ou de l'interligne où se trouve une altération, vous devez ajuster la hauteur de la note correspondante dans toute la partition. Par exemple, s'il y a un dièse à sur la ligne du fa et un dans l'interligne du do, vous devez jouer des fa et des do dièses dans tout le morceau. Cela ajustera automatiquement la tonalité. Elle vous aidera à connaitre la cadence rythmique du morceau. Le chiffrage de mesure qui se trouve au début de la partition indique le nombre de temps par mesure. Le plus commun est 44{\displaystyle {\frac {4}{4}}}, ce qui signifie que chaque mesure comprend quatre noires. Autrement dit, vous devez compter « 1, 2, 3, 4, 1, 2, 3, 4, », etc. pour garder le rythme. D'autres chiffrages donnent d'autres nombres de temps par mesure. Par exemple, 34{\displaystyle {\frac {3}{4}}} indique que chaque mesure comprend trois noires (« 1, 2, 3, 1, 2, 3, », etc.). Les mesures individuelles dans la partition sont séparées par des barres verticales qui coupent la portée. Il y a souvent des numéros au-dessus de chaque portée pour indiquer les numéros de mesures afin qu'un orchestre puisse s'y retrouver facilement. La position d'une note sur la portée détermine sa hauteur. Sa forme (appelée figure de note) indique sa durée. Dans une mesure à quatre temps dont l'unité de temps est la noire (unité la plus commune), une ronde dure une mesure (quatre temps), une blanche dure la moitié d'une mesure (deux temps), la noire dure un quart de mesure (un temps) et ainsi de suite jusqu'à la triple croche. Les figures de notes les plus communes sont les suivantes.    La ronde : O   La blanche : une forme de ronde avec une hampe verticale (ou une forme de noire avec une tête de note blanche)  La noire : ♩  Les silences : un intervalle où il ne faut pas jouer de note, représenté par une pause (un petit trait horizontal gras placé au centre de la mesure) lorsque le silence vaut la durée d'une ronde ou d'une blanche et par un soupir (un petit trait vertical ondulé) lorsque le silence vaut la durée d'une noire
   7  Parler d'un ami commun vous aidera à créer un lien personnel avec cette fille, même si vous ne la connaissez pas bien. Elle se sentira plus à l'aise, car vous ne serez plus un parfait inconnu. Avoir un ami commun vous donnera également un sujet de conversation tout désigné.  Essayez de dire quelque chose comme « J'ai entendu dire que tu as une amie de Carine. Comment est-ce que vous vous êtes rencontrées ? » Ou alors « Tu connais Dan ? On se connait depuis des années ! Il est hilarant, n'est-ce pas ? » Parler d'une expérience commune (qu'il s'agisse de faire du bénévolat ou de grandir dans une ferme) vous aidera à créer une connexion personnelle entre vous deux et à établir le début d'un lien.   Si vous réalisez que vous avez tous deux grandi dans une ferme, vous pourriez dire quelque chose comme « Ce n’est pas vrai ! Moi aussi ! Le pire c'était l'été, quand mon père me réveillait à 5 heures du matin pour que je l'aide dans les champs ! C'était comment pour toi ? » Si vous avez tous les deux faits du bénévolat, vous pourriez dire quelque chose comme « J'ai trouvé que c'était une expérience très enrichissante. Qu'est-ce qui t'a poussé à t'impliquer dans ce projet ? » Poser une question inhabituelle et stimulante à la fille permettra de briser la glace et sera l'occasion pour elle de dire ce qu'elle pense. Elle aura ainsi une chance de s'exprimer et vous ferez bonne impression pour avoir posé une question si intéressante. Gagnant-gagnant !  Essayez quelque chose comme « Si tu pouvais être un animal, quel genre d'animal serais-tu ? » Ou quelque chose comme « Quels sont les 5 endroits que tu veux visiter avant de mourir ? » Ou peut-être « Est-ce que tu oserais faire un saut en parachute ? » Découvrir que vous avez un centre d'intérêt est de l'or en matière de conversation. Cela vous aidera à établir un lien avec la fille. Quel que soit ce centre d'intérêt : la lecture, la course à pied, l'aviron, l'escalade, l'important est de partager quelque chose.  Si vous découvrez que vous aimez tous les deux la course à pied, vous pourriez lui demander quels sont ses itinéraires préférés ou si elle a déjà pensé à s'entrainer pour un marathon. Si vous aimez tous les deux la lecture, demandez-lui quel est son auteur préféré ou ce qu'elle a pensé d'une adaptation cinématographique récente d'un roman très connu. Si c'est un centre d'intérêt très original, demandez-lui comment elle est arrivée à s'y intéresser et comparez vos histoires. Si les choses se passent bien et que vous deux semblez bien vous entendre, il est peut-être temps de devenir un petit peu plus personnel. Souvenez-vous que le but est de montrer qu'elle vous intéresse et que voulez la connaitre, pas la mettre mal à l'aise. Ne posez pas de question qui vous mettraient vous-même mal à l'aise.   Restez positif ! Ne lui demandez pas quelle est sa plus grande peur ou son secret le plus lourd, posez-lui des questions sur ses rêves d'avenir, ses projets pour dans dix ans. Restez-en aux questions auxquelles elle veut bien répondre et restez léger. Essayez de lui poser des questions sur sa famille, essayez quelque chose de simple et de non invasif « Tu as des frères et sœurs ? » Si vous voulez savoir si elle est célibataire, demandez-lui simplement « Tu as quelqu'un dans ta vie en ce moment ? »
   8  Créez un disque en utilisant l'outil fourni par le fabricant de votre ordinateur. Les principaux fabricants d'ordinateurs, comme HP, Dell ou Acer incluent avec Windows des programmes qui permettent de créer des disques de réparation. Si vous avez monté votre propre machine ou si le fabricant n'a pas inclus ce genre d'outils, passez à l'étape suivante pour créer votre propre outil.   HP/Compaq  Prenez 4 DVD-/+R vierges. Vous ne pouvez pas utiliser de DVD-RW. Il est possible que vous n'ayez pas besoin de tous les 4 et vous pouvez également utiliser une clé USB ayant au moins 16 Go d'espace libre. Cliquez sur le bouton Démarrer et tapez « recovery manager ». Sélectionnez « Recovery Manager » dans la liste qui s'affiche. Cliquez « Recovery Media Creation » dans le menu de la fenêtre Recovery Manager. Choisissez le type de média que vous voulez créer. Il peut s'agir d'un DVD ou d'une clé USB. Une fois que vous l'aurez choisi, l'outil vous montrera combien de DVD seront nécessaires ou quelle quantité de mémoire vous devrez libérer sur votre clé USB. Suivez les instructions pour créer les disques. Si vous gravez les disques de réparation, une fenêtre s'affichera chaque fois que vous devrez insérer un disque vierge. Pensez à étiqueter les disques que vous créez pour savoir dans quel ordre les utiliser.    Dell  Lancez « Dell DataSafe Local Backup » dans le dossier « Dell Data » dans la section Tous les programmes du menu Démarrer. Cliquez sur « Backup » et sélectionnez « Create Recovery Media ». Sélectionnez le type de média que vous voulez utiliser. Vous pouvez choisir entre des DVD vierges ou une clé USB. Après avoir choisi, une fenêtre vous dira combien de disques seront nécessaires ou quelle quantité de mémoire devra être libérée sur votre clé USB. Si vous utilisez des disques, vous aurez besoin de DVD+/-R et non RW ou DL. Suivez les instructions pour créer vos disques ou votre lecteur USB. Étiquetez chaque disque créé pour savoir dans quel ordre les utiliser.    Acer/Gateway  Cliquez sur « Acer » dans le menu Démarrer et sélectionnez « Acer eRecovery Management ». Cliquez sur « Backup » et sélectionnez « Create Factory Default Disc ». Insérez votre premier DVD+/-R vierge. Vous aurez besoin de 2 disques vierges. Vous ne pouvez pas utiliser de DVD+/-RW ou DL. Suivez les instructions pour créer les disques de réparation. Étiquetez-les pour savoir dans quel ordre les utiliser. Téléchargez un fichier ISO de Windows 7 ou insérez un disque d'installation de Windows 7. Pour créer votre propre disque d'installation avec tous les pilotes dont vous avez besoin pour votre ordinateur, il vous faudra soit un fichier ISO soit un disque d'installation de Windows 7. Vous pouvez télécharger le fichier ISO de Windows 7 ici. Vous aurez besoin de votre clé de produit pour lancer le téléchargement. Rendez-vous dans la première étape pour plus d'informations. Copiez le contenu du DVD d'installation ou du fichier ISO sur votre disque dur. Vous pouvez créer un nouveau dossier sur votre bureau et glisser-déposer tous les fichiers du disque ou de l'ISO à l'intérieur. Pour ouvrir un fichier ISO, vous aurez besoin de 7-Zip (7-zip.org) ou de WinRAR (rarlab.com) qui sont tous les 2 gratuits. Une fois installé, faites un clic droit sur le fichier ISO et sélectionnez Extraire . Téléchargez et installez le Kit d'Installation Automatisée Windows. Ce programme est nécessaire pour créer votre propre disque d'installation de Windows. Vous pouvez le télécharger gratuitement ici. Le fichier fait environ 1,7 Go et son téléchargement peut prendre un certain temps. Ce programme a été conçu par la communauté d'utilisateurs de Windows pour créer plus facilement un disque d'installation. Vous pouvez le télécharger gratuitement sur nliteos.com et garder tous les paramètres d'installation par défaut . Cliquez sur le bouton Ajouter dans NTLite et sélectionnez le dossier dans lequel vous avez copié les fichiers de Windows. Votre version de Windows 7 est visible dans l'onglet Source. Doublecliquez sur la version de Windows 7 dans l'onglet Source. Convertissez le fichier en image si on vous le demande. Cette opération prend un certain temps. NTLite vous permet d'intégrer des pilotes dans votre installation pour ne pas avoir à les réinstaller après la réparation. Dans le côté droit de la fenêtre, vous verrez une liste de tous les pilotes qui seront automatiquement intégrés. Voyez lesquels sont marqués « manquants ». Téléchargez tous les pilotes « manquants » de votre ordinateur sur le site web du fabricant. Si vous avez la version prémium, vous pouvez simplement les importer depuis votre ordinateur en cliquant sur Importer. Si vous utilisez la version gratuite, il vous faudra télécharger les fichiers sur le site web du fabricant et les installer manuellement.  Rendez-vous sur la page d'assistance de votre ordinateur et cherchez son numéro de modèle. Si vous avez monté vous-même votre machine, vous devrez consulter la page d'assistance pour chacun des composants. Téléchargez tous les pilotes manquants depuis la section Drivers ou Téléchargements. Les fichiers sont au format INF ou EXE. Placez tous les fichiers téléchargés dans un même dossier. Cliquez sur le bouton Ajouter dans la section Drivers. Sélectionnez Dossier pour plusieurs pilotes puis cliquez sur le dossier qui contient tous les pilotes que vous voulez ajouter à votre installation. Tous les fichiers au format INF seront ajoutés. Sélectionnez Post-installation puis Ajouter pour ajouter tous les pilotes au format EXE. Ils se lanceront automatiquement une fois les paramètres de Windows complétés. Si vous le voulez, vous pouvez utiliser NTLite pour automatiser tout le processus d'installation de Windows. Cela vous permet de lancer l'opération et de laisser l'installation personnalisée faire le reste. Cette option est facultative et si vous ne l'autorisez pas, votre installation de Windows se fera normalement.  Allez dans Unattented et sélectionnez Activer. Sélectionnez chaque élément dans la liste et utilisez la liste déroulante à droite pour sélectionner l'option désirée. Cliquez sur Créer un compte local pour créer automatiquement des comptes. Si vous avez la version prémium de NTLite, vous pouvez l'utiliser pour créer automatiquement de nouvelles partitions sur disque. Cliquez sur Appliquer dans le menu de gauche. Quand vous serez satisfait de tous vos réglages, cette section vous permet de finaliser la création de la nouvelle image de réparation. Il vous sera demandé de saisir le nom du nouveau fichier ISO. L'image disque que vous pourrez graver à la fin du processus sera automatiquement créée. La création du nouveau fichier image comprenant tous les pilotes va commencer. L'opération dure environ 20 minutes. Faites un clic droit sur le fichier ISO et sélectionnez Graver l'image disque. Assurez-vous qu'il y ait un DVD vierge dans votre graveur. Windows va graver le fichier ISO dans le DVD et créer votre disque de réparation personnalisé.
   9  Pour ce que nous savons de nos jours, les chats peuvent s'exprimer avec 16 types de miaulements. Pour améliorer votre communication, vous devez prêter beaucoup d'attention aux sonorités que votre ami félin produit.  Un miaulement court exprime normalement que vous êtes le (la) bienvenu. Des miaulements répétitifs montrent de l'excitation. Un miaulement ayant un ton moyen est généralement une demande, le plus souvent liée à la nourriture. Un miaulement profond signale surement du mécontentement. Un miaulement aigu peut indiquer de la douleur ou du mécontentement. Un miaulement long mrrrrowww exprime un besoin . Par nature, les chatons commencent à miauler dans la vie pour attirer l'attention de leur mère. Ils font cela en réalisant que vous ne répondez pas à d'autres signaux. Les miaulements expriment souvent un besoin ou un désir, mais ils peuvent également communiquer bien d'autres choses.  La plupart du temps, le miaulement de votre matou est une demande. Il vous dit par exemple qu'il veut de l'eau, de la nourriture, de l'affection ou la possibilité de changer de pièce, d'endroit. Un miaulement peut aussi être un signal de détresse. Les chats vont miauler quand ils sont stressés, quand ils ont trop chaud, quand ils se sentent seuls, quand ils souffrent, ou encore, quand ils sont de mauvaise humeur. En grandissant, ils vont s'exprimer de manière plus fréquente. Parfois, un miaulement peut simplement être le signe qu'il vous souhaite la bienvenue . Les chats communiquent principalement avec le langage corporel. Ils ont un système de communication très élaboré composé de signaux qu'ils expriment grâce à leur jolie queue !  Si votre matou vous regarde dans les yeux tout en faisant un lent et long battement des paupières, il vous montre de l'affection. Vous pouvez comparer cela à un bisou. Si votre ami s'approche de vous avec la queue levée, c'est une forme de vous montrer qu'il est heureux de vous voir et de vous souhaiter la bienvenue. Ceci peut être comparé à une solide poignée de mains . Lorsque le pelage de votre chat est dressé, il exprime un sentiment d'insécurité, de la peur ou encore du mécontentement. Si vous voyez que la queue de votre ami est entre ses jambes et basse, il montre qu'il est anxieux et mal à l'aise. S'il remue la queue, il exprime qu'il n'est pas content du tout. Plus la queue se déplace rapidement, plus votre chat est de mauvaise humeur .
  10  Une méduse, même morte, reste toxique et son poison peut toujours provoquer une piqure. Certaines espèces ont des tentacules très longs, comme les méduses portugaises dont les membres peuvent être de près de dix mètres de long. Il est donc préférable de rester à l'écart des méduses échouées .  Il existe toutes sortes de méduses et elles peuvent ressembler à des sacs en plastique ou à d'autres débris quand elles sont échouées sur une plage. Il est préférable de ne pas y toucher, si vous ne savez pas au juste s'il s'agit d'une méduse ou non. Prévenez un sauveteur ou un responsable de la plage si vous avez remarqué une méduse sur le rivage, afin que la créature soit enlevée en toute sécurité par une personne qui sait comment la manipuler. Les sauveteurs en mer sont formés pour venir au secours des baigneurs dans des situations diverses, y compris les attaques de méduses. Ils sont aussi en mesure de vous prévenir quand ils ont remarqué la présence de méduses. Vous pouvez perturber et faire déguerpir quelques méduses ou d'autres créatures qui peuvent vous piquer, si vous remuez le courant en marchant dans des eaux peu profondes . Vous pouvez souvent voir la méduse dans l'eau. Elle peut ressembler à un sac en plastique ou à un ballon qui flotte à la surface de l'eau et peut se présenter en grappes . Retournez calmement sur le rivage pour éviter une piqure si vous voyez une méduse dans l'eau.  Prévenez un sauveteur ou un responsable de la plage quand vous avez remarqué une méduse dans l'eau, afin que les autres baigneurs soient prévenus aussi.
  11  Ce qu'il y a de bien avec la lecture est évidemment de pouvoir le faire quasiment n'importe quand et n'importe où. Mais le fait de choisir un moment précis de la journée pour le faire peut vous aider, si ce que vous lisez ne vous enthousiasme pas vraiment. Mettez en route une sonnerie de réveil ou créez-vous une sonnerie de rappel sur le téléphone pour vous signaler votre temps de lecture et la fin de la période, cherchez votre livre et allez-y.   Prévoyez des périodes de temps assez gérables. Prévoyez de petites pauses pour vous lever et vous étirer environ toutes les vingt minutes si vous lisez d'une traite une heure ou plus. Variez votre menu. Vous n'avez pas besoin de toujours lire au même endroit. Vous pouvez le faire dehors, si le temps s'y prête. Vous pouvez aussi vous rendre dans une bibliothèque ou un café près de chez vous pour changer, si vous en avez assez de lire chez vous. Vous pouvez toujours trouver une occasion de lire. Vous pourriez être coincé dans la salle d'attente du dentiste pendant une demi-heure ou arriver vingt minutes en avance à un cours de natation. Vous pouvez tirer parti de ces périodes d'attente si vous avez emporté votre livre et avancer un peu dans votre lecture.  Les livres électroniques vous facilitent encore la tâche, étant donné que vous pouvez lire votre ouvrage directement sur votre téléphone mobile. Le meilleur moyen d'aller au bout de votre liste de lectures est de lire un peu chaque jour, indépendamment de la nature de votre planning. Cela va non seulement vous aider à avancer dans votre liste, mais cela vous évitera aussi de perdre le fil de ce que vous étiez en train de lire et vous épargner une relecture fastidieuse. Il est probable qu'on vous interrogera sur vos lectures ou qu'on vous demandera de faire un devoir sur table à ce sujet, si vous devez aller au bout d'une liste de lectures. Le fait de prendre des notes vous aidera à vous souvenir de ce que vous avez lu. Vous n'avez sans doute pas besoin de faire des notes détaillées, mais plutôt vous procurer quelques informations de base comme le nom des personnages principaux et leurs caractéristiques, un résumé du sujet de l'histoire pour chaque chapitre . Votre emploi du temps risque d'être rapidement chargé et quasiment impossible à suivre si vous manquez l'un de vos délais. Si vous êtes à la traine, annulez d'autres activités si c'est nécessaire et rattrapez votre retard le plus vite possible. Vous vous en féliciterez à la fin du mois d'aout (ou de septembre, si vous entrez en fac). Ne cherchez pas à deviner le sens de mots que vous ne comprenez pas. Vérifiez-les dans un dictionnaire ! Recherchez autour de vous des personnes qui ont déjà lu ce livre, si vous avez du mal à comprendre l'intrigue de l'histoire ou lisez une analyse de lecture de l'œuvre dans une bibliothèque ou en ligne.  Le fait de lire un résumé ou une analyse de livre peut vous aider, mais cela ne devrait pas remplacer la lecture de cet ouvrage. Rencontrez celui ou celle qui est d'accord pour commenter le livre et comparer ses notes aux vôtres, si la personne lit le même livre que vous. Envisagez de regarder ensemble le film le plus récent qui s'est inspiré de ce livre quand vous avez tous deux fini de le lire, si vous pouvez en trouver un. Vous pouvez ensuite voir ensemble en quoi le film diffère du livre et quelle est votre version préférée. Vous pouvez aussi disserter sur les détails de l'épreuve que vous venez de passer. Il peut être difficile d'être enthousiaste au sujet de l'obligation de lire de façon compulsive, mais vous ne devriez pas oublier que la plupart des livres figurant sur votre liste ont été choisis parce qu'ils sont vraiment bons. Ignorez le fait que vous êtes obligé de lire tous les livres de votre liste et essayez d'apprécier cette expérience. Vous pourriez être agréablement surpris par le côté amusant et intéressant de ce devoir de vacances.
  12  Il apportera un peu de vie au texte. En ce qui concerne le format, une présentation informelle est semblable à une description professionnelle, la différence majeure étant le niveau de langue. Dans un contexte non formel, faites ressortir votre personnalité à l’aide d’humour, de traits d’esprit et de mots évocateurs . Contrairement à une écriture formelle, vous pouvez employer des contractions, des points d’exclamation et d’autres éléments non formels. Toutefois, assurez-vous que votre grammaire est correcte et évitez d’employer de l’argot et de gros mots. Comme pour une description formelle, expliquez qui vous êtes et donnez des informations clés sur vous-même. S’il y a des consignes, respectez-les. Vérifiez également si vous devez écrire à la première ou à la troisième personne. Si vous pouvez faire ce que vous voulez, choisissez ce qui vous semble le plus naturel. En général, il est préférable d’employer la première personne sur les réseaux sociaux . Vous pouvez écrire quelque chose comme : « Marion Dubois est coach de vie depuis plus de 10 ans. Sa passion est d’aider ses clients à vivre leur vie le mieux possible. Pendant son temps libre, elle aime faire des câlins à ses chats et faire de la randonnée avec son époux, Simon. » Parlez de loisirs, de centres d’intérêt ou d’autres éléments qui permettront aux lecteurs de mieux vous connaitre. Vous pouvez parler de vos animaux de compagnie, de votre famille, d’un talent particulier ou d’une expérience en rapport avec le but de votre présentation . Si vous rédigez une présentation pour un article de cuisine que vous avez écrit, vous pouvez écrire quelque chose comme : « Ma passion pour la cuisine remonte à mon enfance, lorsque ma grand-mère m’apprenait ses vieilles recettes de famille. Je me suis rendu compte que la nourriture était intimement liée à la famille, à l’histoire et à la tradition. » En général, respectez une limite de cent à deux-cents mots. Vous ne rédigez pas une dissertation ou une autobiographie. Tâchez de rester concis(e). Dans la plupart des cas, un paragraphe court composé de trois à cinq phrases et comprenant cent à deux-cents mots en tout est suffisant pour fournir les informations clés . Si vous ne savez pas quelle longueur est appropriée, cherchez des consignes ou des exemples de présentations que d’autres personnes ont écrites pour vous servir de modèles. Par exemple, si vous avez écrit un article qui sera publié dans une revue et que vous devez vous présenter, cherchez des présentations d’autres auteurs.
  13  Il est tout à fait possible pour une personne de faire des recherches, planifier, filmer et modifier un documentaire seule, surtout si l'étendue du documentaire est relativement resserrée ou intimiste. Cependant, beaucoup de personnes trouveront cette approche trop difficile ou penseront que c'est de l'amateurisme. Pensez à recruter une aide expérimentée pour votre documentaire, surtout si vous vous attaquez à un sujet ambitieux ou si vous voulez que votre film ait l'air abouti et professionnel. Pour obtenir de l'aide, vous pouvez recruter des amis et des connaissances qualifiées qui feront de la publicité pour votre projet en distribuant des prospectus ou en postant des messages en ligne ou bien contacter une agence. Voici quelques types de professionnels qu'il faut penser à recruter :  des caméramans des régisseurs éclairagistes des scénaristes des chercheurs des monteurs des acteurs (pour les séquences écrites) des techniciens audios des consultants techniques Lorsque vous recrutez votre équipe, cherchez des gens qui partagent les mêmes valeurs lorsqu'il s'agit de la problématique de votre documentaire. Pensez aussi à engager une jeune équipe inspirée et au fait du marché et du public que vous visez.  Discutez toujours avec votre cadreur et les autres créatifs qui sont impliqués dans votre documentaire. Cela vous aidera à en faire le fruit d'un effort collectif partageant la même vision. Travailler dans un environnement collaboratif signifie que votre équipe trouvera souvent de nouvelles idées en contribuant au projet sous des angles que vous pourriez avoir négligés. Les réalisateurs de documentaires sérieux doivent au moins comprendre comment leurs films sont produits, mis en scène, tournés et montés, même s'ils ne peuvent pas tout faire eux-mêmes. Si vous n'êtes pas familiarisé avec les procédés techniques qui se cachent derrière la réalisation, vous trouverez surement que cela vaut la peine d'étudier la réalisation avant de tourner votre documentaire. Beaucoup d'universités dispensent des cours de réalisation, mais vous pouvez aussi acquérir de l'expérience pratique en travaillant sur des tournages et en passant devant ou derrière la caméra. Même si la plupart des réalisateurs ont suivi un cursus cinématographique, les connaissances pratiques peuvent supplanter des connaissances filmiques classiques. Par exemple, le comédien Louis C.K qui a réalisé des films et pour la télévision a acquis une expérience de réalisateur très tôt en travaillant au sein d'une chaine publique locale . Essayez d'utiliser le meilleur matériel possible. Empruntez l'équipement que vous ne pouvez pas vous payer et utilisez vos contacts pour avoir accès à du matériel. Vous ne devez pas forcément exactement savoir de quoi votre documentaire aura l'air avant de commencer à le tourner. Vous découvrirez probablement certaines choses qui changeront vos plans et vous offriront de nouvelles pistes d'investigation pendant que vous filmerez. Cependant, vous devriez vraiment avoir un plan avant de commencer le tournage. Avoir un plan vous laissera du temps pour programmer les interviews, gérer les conflits d'emploi du temps, etc. Votre plan doit inclure ce qui suit.   Les personnes que vous voulez interviewer. Prenez contact avec ces personnes le plus tôt possible pour programmer des entretiens. Les évènements particuliers que vous voulez enregistrer. Prévoyez de vous rendre et de revenir de ces évènements, d'acheter des billets si nécessaire et d'obtenir l'autorisation de filmer auprès des organisateurs des évènements. Les textes, images, dessins, musiques ou autres documents que vous voulez utiliser. Obtenez la permission de les utiliser auprès des créateurs avant de les ajouter à votre documentaire. Les reconstitutions que vous voulez filmer. Cherchez les acteurs, les accessoires et les lieux de tournage longtemps à l'avance.
  14  En écrivant l'histoire de votre vie, vous en apprendrez beaucoup sur qui vous êtes et sur la façon dont vous appréhendez les défis, les joies, les opportunités et les difficultés qui jalonnent votre route. En écrivant votre histoire personnelle, vous aurez un aperçu de ce que vous avez appris au cours des expériences que vous avez traversées et comment ces expériences vous ont affectées. De cette façon, vous pourrez analyser comment vos expériences ont fait de vous celui que vous êtes et comment elles ont défini vos valeurs, votre comportement, vos croyances, vos défauts, vos idées préconçues, vos réactions et la façon dont vous interagissez avec votre environnement. Une fois que vous aurez écrit l'histoire de votre vie, analysez-vous en vous posant certaines questions.  Quels sont les thèmes récurrents de votre histoire ? Êtes-vous toujours celui que l'on sauve ou celui qui sauve les autres ? Votre thème dominant est-il l'impuissance ou au contraire le pouvoir ? Votre histoire est-elle une histoire d'amour, une comédie, un drame ou autre chose ? Si vous deviez donner un titre à votre histoire, quel serait-il ? Divisez votre histoire en chapitres. Pourquoi les chapitres sont-ils définis de la sorte ? Qu'est-ce qui a changé ? Qu'avez-vous appris ? Quels sont les titres de ces chapitres ? Avez-vous utilisé un qualificatif particulier pour vous définir ? L'avez-vous fait pour d'autres personnes ? Que signifient pour vous ces qualificatifs et que disent-ils de votre vision de vous-même, des autres et du reste du monde ? Quels types de mots utilisez-vous pour vous décrire, pour décrire les autres, pour décrire le reste du monde ? Que disent ces mots de votre histoire et de la façon dont vous l'avez vécue ? Une fois que vous aurez écrit votre histoire, vous devez décider de ce que cela signifie. Ce qui est intéressant dans le fait d'écrire votre propre histoire, ce que l'on appelle la thérapie narrative, c'est que cela vous montre que ce que vous pensez est important ou crucial dans votre existence. Cela met en lumière les moments de votre vie qui vous semblent importants ou qui valent la peine d'être notés. Cela montre également la façon dont vous vous percevez et dont vous percevez la trajectoire que votre vie a empruntée. Par exemple, si vous écrivez votre vie comme un drame, c'est peut-être que votre histoire vous semble dramatique et intense. Si vous l'avez écrite comme une comédie, c'est peut-être qu'elle vous parait drôle et gaie. Si vous l'avez écrite comme une histoire d'amour, vous êtes peut-être un incorrigible romantique qui vit peut-être le grand amour ou qui espère le vivre un jour. Même lorsque vous avez suivi toutes ces étapes, vous devez prendre conscience que cela prendra du temps. Il est important de comprendre que si vous vous connaissez déjà mieux ou si vous vous êtes bien analysé, vous devrez continuer à faire cela tout au long de votre vie. Votre personnalité et vos croyances peuvent changer avec le temps.
  15  Dans un saladier de taille moyenne, versez le mélange pour pouding, puis ajoutez le lait et la vanille. Fouettez les ingrédients jusqu'à ce qu'ils soient bien mélangés et que le mélange commence à épaissir.  Vous pouvez remplacer l'extrait de vanille par de l'extrait d'amandes pour obtenir une saveur plus sophistiquée.  Vous pouvez aussi remplacer le pouding à la vanille par tout autre arôme, comme chocolat, caramel, cheese-cake, etc.


~~~